## Proyecto Music Stream

**Para la profesora:** 👩🏻‍🏫
Explicar cual es el objetivo del proyecto y cuales son las fases por las que pasará el proyecto.

### Objetivo

1. Crear una bases de datos para almacenar información de canciones y artistas. 
2. Extraer información de la base de datos para responder preguntas sobre la música almacenada.

### Fases del proyecto 

![](./assets/esquema_de_fases.png)

**Fase 1: Extraer Datos**
1. Extraer información de Api Spotify
2. Extraer información de API Last FM
3. Guardar esa información en CSV

**Fase 2:Bases de datos**
1. Diseñar la Base de Datos
2. Crear la bases de datos
3. Insertar los valores en la bases de datos
   
**Fase 3:Extraer información**
1. Extraer información de la base de datos para responder preguntas sobre la música almacenada.
   

## Comencemos... Fase 1.1 Extraer información de Api Spotify


In [1]:
## - Explicar como usar la libreria spotipy 
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
import pandas as pd
import requests

In [3]:
# Coloca tus credenciales aquí
CLIENT_ID = 'a018711755a146b2a678a93741dc6041'
CLIENT_SECRET = '51fa44e4b57343f19c810b9a75089da6'

# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

### Como usar el método de search de Spotipy

In [4]:
datos = sp.search(q="genre:rock year:2020", type='track', limit=50, offset=0)

In [5]:
genres = ["pop" , "rock","jazz","hip hop"]
year_start = 2002
year_end = 2007

lista_canciones_albumes = [] 
lista_artistas = []

In [6]:
datos['tracks']['items'][0]['album']['type']


'album'

In [ ]:
for genre in genres: 
    datos = sp.search(q=f"genre:{genre} year:{year_start}-{year_end}", type='track,album', limit=50, offset=951)
    datos
    for item in datos['tracks']['items']: 
        
        lista_canciones_albumes.append(
            {
                "nombre_cancion":item['name'],
                "nombre_artista":item['artists'][0]['name'],
                'genre':genre,                 
                "type":  item['album']['type'],
                "year": item['album']['release_date'][:4],
                "id": item['id']
            }
        )
        lista_artistas.append(
            {
                "nombre_artista":item['artists'][0]['name'],
            }
        )
   

In [38]:
len(lista_artistas)

3850

In [39]:
len(lista_canciones_albumes)

3850

In [40]:
df_lista_artistas = pd.DataFrame(lista_artistas)

In [41]:
lista_artistas_limpia = set(df_lista_artistas["nombre_artista"])


In [43]:

df_lista_canciones_albumes = pd.DataFrame(lista_canciones_albumes)
df_lista_canciones_albumes.to_csv("canciones.csv", index= False)


In [44]:
df_artistas_limpia = pd.DataFrame(lista_artistas_limpia)
df_artistas_limpia.to_csv("artistas.csv", index = False)

In [ ]:
# Ahora que ya tenemos los dos archivos csv con la informacion de artistas y de canciones y albumes, 
# vamos a la API de Lastfm para sacar la informacion de cada artista.


In [45]:
# API Key obtenida de last.fm
API_KEY = 'bb92344bdb091e7c718014c1dcd0e28d'  
BASE_URL = 'http://ws.audioscrobbler.com/2.0/'

In [48]:
datos_artistas = []

In [49]:
for artista in lista_artistas_limpia:
    params_info = {
        "method": "artist.getInfo", 
        "api_key": API_KEY,
        "format": "json", 
        "artist": artista
    }
    try:
        response_info = requests.get(BASE_URL, params = params_info) #consulta de la API
        if response_info.status_code !=200:
            print(f"error en la solicitud para {artista}: {response_info.status_code}")
            continue
        data_json = response_info.json()
        resumen = data_json['artist']['bio']['summary']
        datos_artistas.append({"artista": artista, "resumen": resumen})
    except KeyError:
        print(f"Falta información para el artista: {artista}")
    except Exception as e:
        print(f"Ocurrió un error inesperado con {artista}: {e}")
        
    

Ocurrió un error inesperado con Luis Salinas: Expecting value: line 1 column 1 (char 0)


In [50]:
df_datos_artistas = pd.DataFrame(datos_artistas)
df_datos_artistas.to_csv("datos_artistas.csv", index = False)